In [20]:
!pip install folium
!pip install pandas
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import re


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
files = [
    "Waze heavy traffic.csv",
    "HAZARD_ON_ROAD_TRAFFIC_LIGHT_FAULT.csv",
    "Waze pot_hole.csv",
    "HAZARD_WEATHER_FLOOD.csv",
    "Waze accident minor.csv",
    "Waze accident major.csv",
    "Waze stand still traffic.csv"
]

dfs = []

for f in files:
    df = pd.read_csv(f, low_memory=False)
    dfs.append(df)

waze_all = pd.concat(dfs, ignore_index=True)

In [22]:
waze_all["longitude"] = waze_all["Location"].str.extract(r"Point\(([\d\.]+)")
waze_all["latitude"] = waze_all["Location"].str.extract(r"Point\([\d\.]+\s([\d\.]+)")

waze_all["longitude"] = waze_all["longitude"].astype(float)
waze_all["latitude"] = waze_all["latitude"].astype(float)

In [23]:
villes_service_commun = [
    "Palaiseau","Orsay","Villejust","Ballainvilliers","Verrières-le-Buisson",
    "La Ville-du-Bois","Les Ulis","Saclay","Wissous","Villebon-sur-Yvette",
    "Saulx-les-Chartreux","Villiers-le-Bâcle","Linas","Vauhallan","Saint-Aubin",
    "Longjumeau","Marcoussis","Nozay","Epinay-sur-Orge","Igny"
]

waze_all["City"] = waze_all["City"].fillna("Inconnue")
waze_all = waze_all[waze_all["City"].isin(villes_service_commun)]

In [24]:
mois = {
    "janv": "Jan", "févr": "Feb", "mars": "Mar", "avr": "Apr",
    "mai": "May", "juin": "Jun", "juil": "Jul", "août": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "déc": "Dec"
}

def clean_date(d):
    if pd.isna(d):
        return pd.NaT
    d = str(d).lower()
    for fr, en in mois.items():
        d = re.sub(fr + r"\.?", en, d)
    return pd.to_datetime(d, errors="coerce")

waze_all["Date"] = waze_all["Date"].apply(clean_date)
waze_all["annee"] = waze_all["Date"].dt.year

In [25]:
scenario_map = {
    "JAM_HEAVY_TRAFFIC": "Trafic dense",
    "JAM_STAND_STILL_TRAFFIC": "Trafic à l’arrêt",
    "ACCIDENT_MINOR": "Accident léger",
    "ACCIDENT_MAJOR": "Accident grave",
    "HAZARD_ON_ROAD_POT_HOLE": "Nid-de-poule",
    "HAZARD_ON_ROAD_TRAFFIC_LIGHT_FAULT": "Panne de feu tricolore",
    "HAZARD_WEATHER_FLOOD": "Inondation"
}

waze_all["scenario"] = waze_all["Subtype"].map(scenario_map)
waze_all = waze_all.dropna(subset=["scenario"])

In [26]:
villes = sorted(waze_all["City"].unique())
annees = sorted(waze_all["annee"].dropna().unique())

In [60]:
ICONES = {
    "Trafic dense": "https://img.icons8.com/color/48/traffic-jam.png",
    "Trafic à l’arrêt": "https://img.icons8.com/color/24/traffic-jam.png",
    "Accident léger": "https://img.icons8.com/color/48/car-crash.png",
    "Accident grave": "https://img.icons8.com/color/48/car-accident.png",
    "Nid-de-poule": "https://img.icons8.com/color/48/road-worker.png",
    "Panne de feu tricolore": "https://img.icons8.com/color/48/traffic-light.png",
    "Inondation": "https://img.icons8.com/color/48/floods.png"
}

In [64]:
m = folium.Map(
    location=[48.70, 2.20],
    zoom_start=11,
    tiles="CartoDB positron"
)

In [65]:
layers = {}

for ville in villes:
    for annee in annees:
        key = f"{ville} – {annee}"
        layers[key] = folium.FeatureGroup(name=key, show=False)

In [66]:
for _, row in waze_all.iterrows():
    key = f"{row['City']} – {row['annee']}"

    if key in layers:

        icon = folium.CustomIcon(
            ICONES[row["scenario"]],
            icon_size=(28, 28)
        )

        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            icon=icon,
            popup=f"""
            <b>Scénario :</b> {row['scenario']}<br>
            <b>Ville :</b> {row['City']}<br>
            <b>Année :</b> {row['annee']}<br>
            <b>Rue :</b> {row['Street']}
            """
        ).add_to(layers[key])

In [67]:
legend_html = """
<div style="
position: fixed;
bottom: 30px;
left: 10px;
z-index: 9999;
background: white;
padding: 10px;
border-radius: 6px;
box-shadow: 0 0 10px rgba(0,0,0,0.3);
">
<b>Légende</b><br>
<img src="https://img.icons8.com/color/48/traffic-jam.png"> Trafic dense<br>
<img src="https://img.icons8.com/color/24/traffic-jam.png"> Trafic à l’arrêt<br>
<img src="https://img.icons8.com/color/48/car-crash.png"> Accident léger<br>
<img src="https://img.icons8.com/color/48/car-accident.png"> Accident grave<br>
<img src="https://img.icons8.com/color/48/road-worker.png"> Nid-de-poule<br>
<img src="https://img.icons8.com/color/48/traffic-light.png"> Panne de feu tricolore<br>
<img src="https://img.icons8.com/color/48/floods.png"> Inondation
</div>
"""

In [68]:
for layer in layers.values():
    layer.add_to(m)

In [69]:
folium.LayerControl(collapsed=False).add_to(m)

In [11]:
filter_html = f"""
<div style="
position: fixed;
top: 10px;
left: 10px;
z-index: 9999;
background: white;
padding: 10px;
border-radius: 6px;
box-shadow: 0 0 10px rgba(0,0,0,0.3);
">
<b>Filtres</b><br><br>

Ville :
<select id="ville">
<option value="all">Toutes</option>
{''.join([f'<option value="{v}">{v}</option>' for v in villes])}
</select><br><br>

Année :
<select id="annee">
<option value="all">Toutes</option>
{''.join([f'<option value="{a}">{a}</option>' for a in annees])}
</select>
</div>

<script>
var markers = [];

map.eachLayer(function(layer) {{
    if (layer instanceof L.CircleMarker) {{
        markers.push(layer);
    }}
}});

function filterMarkers() {{
    var ville = document.getElementById("ville").value;
    var annee = document.getElementById("annee").value;

    markers.forEach(function(marker) {{
        var visible =
            (ville === "all" || marker.options.ville === ville) &&
            (annee === "all" || marker.options.annee == annee);

        if (visible) {{
            marker.setStyle({{opacity: 1, fillOpacity: 0.7}});
        }} else {{
            marker.setStyle({{opacity: 0, fillOpacity: 0}});
        }}
    }});
}}

document.getElementById("ville").onchange = filterMarkers;
document.getElementById("annee").onchange = filterMarkers;
</script>
"""


In [70]:
m.get_root().html.add_child(folium.Element(legend_html))

In [71]:
m.save("carte_waze_service_commun.html")